+ Day, Hour, Minute
+ DHI
+ DNI
+ WS
+ RH
+ T
<br>

+ TARGET

+ 학습 데이터로 모델을 학습시키고,
+ 총 6일간의 테스트 데이터를 활용해서
+ 2일동안의 Target을 쿼터별로 예측

In [1]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
DATA_PATH = './data/'

train = pd.read_csv(DATA_PATH + 'train/train.csv')

col_num= train.shape[-1]
row_num = train.shape[0]

In [3]:
sequence_level = 6
target_level = 2

def make_data_sequence(train, sequence_level, target_level):
    iter_num = row_num//(48 * sequence_level) # 6일치
    sequence_len = 48 * sequence_level
    target_sequence_len = 48 * target_level
    
    data = np.empty((row_num//sequence_len, sequence_len, col_num))
    label = np.empty((row_num//sequence_len, target_sequence_len,1))

    for i in range(iter_num):
        if((i + 1)*288 < len(train)):
            data[i] = train.iloc[i*288:(i+1)*288, :]
            # loc는 a:b에서 b도 포함함
            label[i] = train.loc[(i+1)*288:((i+1)*288) + target_sequence_len - 1, 'TARGET'].to_numpy().reshape(-1, 1)
        
    return data, label 

data, label = make_data_sequence(train, sequence_level, target_level)
label = label[:, :, 0]

In [4]:
x_train = data[:152]; x_val = data[152:]
y_train = label[:152]; y_val = label[152:]

## Modeling

In [8]:
from tensorflow.keras.layers import GRU, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MAE
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
from tensorflow.keras import backend as K 

def q_loss(q):
    def quantile_loss(y_true, y_pred):
        e = K.cast(y_true - y_pred, dtype = 'float32')
        
        return K.mean(K.max([e * q, e * (q - 1.0)]))
    
    return quantile_loss

In [12]:
def get_model():
    inputs = Input(shape=(288, 9))
    
    x = GRU(units = 25, return_sequences = True, activation = 'tanh')(inputs)
    x = GRU(units = 50, return_sequences = False, activation = 'tanh')(x)
    x = Dense(96)(x)
    
    model = Model(inputs = inputs, outputs = x)
    
    return model

model = get_model()

model.compile(optimizer = Adam(learning_rate = 3e-4),
             loss = q_loss(q=0.1))

In [13]:
model.fit(x_train, y_train,
         epochs = 500,
         batch_size = 16,
         validation_data = (x_val, y_val))

Epoch 1/500
10/10 [==============================] - 1s 83ms/step - loss: 9.6977 - val_loss: 8.3662
Epoch 2/500
10/10 [==============================] - 0s 30ms/step - loss: 9.6730 - val_loss: 8.3593
Epoch 3/500
10/10 [==============================] - 0s 26ms/step - loss: 9.6770 - val_loss: 8.3532
Epoch 4/500
10/10 [==============================] - 0s 25ms/step - loss: 9.6723 - val_loss: 8.3432
Epoch 5/500
10/10 [==============================] - 0s 25ms/step - loss: 9.6714 - val_loss: 8.3384
Epoch 6/500
10/10 [==============================] - 0s 24ms/step - loss: 9.5646 - val_loss: 8.3347
Epoch 7/500
10/10 [==============================] - 0s 29ms/step - loss: 9.5918 - val_loss: 8.3260
Epoch 8/500
10/10 [==============================] - 0s 24ms/step - loss: 9.6783 - val_loss: 8.3163
Epoch 9/500
10/10 [==============================] - 0s 24ms/step - loss: 9.6614 - val_loss: 8.3072
Epoch 10/500
10/10 [==============================] - 0s 24ms/step - loss: 9.6248 - val_loss: 8.2959

10/10 [==============================] - 0s 30ms/step - loss: 9.0138 - val_loss: 7.7531
Epoch 83/500
10/10 [==============================] - 0s 22ms/step - loss: 9.0857 - val_loss: 7.7474
Epoch 84/500
10/10 [==============================] - 0s 22ms/step - loss: 9.0208 - val_loss: 7.7409
Epoch 85/500
10/10 [==============================] - 0s 24ms/step - loss: 8.9883 - val_loss: 7.7368
Epoch 86/500
10/10 [==============================] - 0s 33ms/step - loss: 9.0793 - val_loss: 7.7309
Epoch 87/500
10/10 [==============================] - 0s 23ms/step - loss: 8.9947 - val_loss: 7.7259
Epoch 88/500
10/10 [==============================] - 0s 32ms/step - loss: 9.0569 - val_loss: 7.7197
Epoch 89/500
10/10 [==============================] - 0s 23ms/step - loss: 8.9842 - val_loss: 7.7139
Epoch 90/500
10/10 [==============================] - 0s 24ms/step - loss: 9.0707 - val_loss: 7.7106
Epoch 91/500
10/10 [==============================] - 0s 28ms/step - loss: 9.0494 - val_loss: 7.7061
Epo

10/10 [==============================] - 0s 28ms/step - loss: 8.7261 - val_loss: 7.4746
Epoch 163/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6851 - val_loss: 7.4900
Epoch 164/500
10/10 [==============================] - 0s 21ms/step - loss: 8.6635 - val_loss: 7.5049
Epoch 165/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6463 - val_loss: 7.5251
Epoch 166/500
10/10 [==============================] - 0s 22ms/step - loss: 8.7198 - val_loss: 7.5343
Epoch 167/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6766 - val_loss: 7.5493
Epoch 168/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6536 - val_loss: 7.5551
Epoch 169/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6465 - val_loss: 7.5847
Epoch 170/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6594 - val_loss: 7.6087
Epoch 171/500
10/10 [==============================] - 0s 22ms/step - loss: 8.6755 - val_loss: 7

10/10 [==============================] - 0s 22ms/step - loss: 8.4666 - val_loss: 9.5835
Epoch 243/500
10/10 [==============================] - 0s 22ms/step - loss: 8.4403 - val_loss: 9.6199
Epoch 244/500
10/10 [==============================] - 0s 22ms/step - loss: 8.4372 - val_loss: 9.6642
Epoch 245/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3800 - val_loss: 9.6818
Epoch 246/500
10/10 [==============================] - 0s 22ms/step - loss: 8.4276 - val_loss: 9.6978
Epoch 247/500
10/10 [==============================] - 0s 21ms/step - loss: 8.4558 - val_loss: 9.7609
Epoch 248/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3990 - val_loss: 9.7830
Epoch 249/500
10/10 [==============================] - 0s 22ms/step - loss: 8.4594 - val_loss: 9.8048
Epoch 250/500
10/10 [==============================] - 0s 22ms/step - loss: 8.4513 - val_loss: 9.8564
Epoch 251/500
10/10 [==============================] - 0s 24ms/step - loss: 8.4062 - val_loss: 9

10/10 [==============================] - 0s 22ms/step - loss: 8.3362 - val_loss: 9.9353
Epoch 323/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3800 - val_loss: 9.9544
Epoch 324/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3789 - val_loss: 9.9859
Epoch 325/500
10/10 [==============================] - 0s 21ms/step - loss: 8.4240 - val_loss: 10.0090
Epoch 326/500
10/10 [==============================] - 0s 23ms/step - loss: 8.3616 - val_loss: 9.9262
Epoch 327/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3534 - val_loss: 9.9048
Epoch 328/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3993 - val_loss: 9.9257
Epoch 329/500
10/10 [==============================] - 0s 22ms/step - loss: 8.4670 - val_loss: 9.9616
Epoch 330/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3862 - val_loss: 9.9079
Epoch 331/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3335 - val_loss: 

Epoch 402/500
10/10 [==============================] - 0s 21ms/step - loss: 8.4122 - val_loss: 10.2143
Epoch 403/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3860 - val_loss: 10.2268
Epoch 404/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3720 - val_loss: 10.2394
Epoch 405/500
10/10 [==============================] - 0s 23ms/step - loss: 8.3621 - val_loss: 10.2445
Epoch 406/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3296 - val_loss: 10.2540
Epoch 407/500
10/10 [==============================] - 0s 21ms/step - loss: 8.4234 - val_loss: 10.2663
Epoch 408/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3421 - val_loss: 10.2782
Epoch 409/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3483 - val_loss: 10.2822
Epoch 410/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3401 - val_loss: 10.2845
Epoch 411/500
10/10 [==============================] - 0s 22ms/step - los

10/10 [==============================] - 0s 22ms/step - loss: 8.2663 - val_loss: 10.4185
Epoch 482/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3829 - val_loss: 10.4423
Epoch 483/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3843 - val_loss: 10.4524
Epoch 484/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3106 - val_loss: 10.3431
Epoch 485/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3956 - val_loss: 10.3387
Epoch 486/500
10/10 [==============================] - 0s 23ms/step - loss: 8.4098 - val_loss: 10.3852
Epoch 487/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3597 - val_loss: 10.4182
Epoch 488/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3775 - val_loss: 10.4440
Epoch 489/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3267 - val_loss: 10.4638
Epoch 490/500
10/10 [==============================] - 0s 22ms/step - loss: 8.3607 - va